# Using PhiData Agent to work as a Data Analyst


In this notebook, we will use **PhiData**, a framework for defining and running AI agents for specific tasks. This example, we can see how a structured data (in form of a table) can be utilised by agent to get insights.

## Steps Overview
1. Define a tool to access a data source (CSV)
2. Define an agent with LLM and the Data tool to perform data analysis.
3. Initate Agent in interactive mode

---
### Prerequisites
- Install the `phidata` library.
- Obtain API keys for the LLM model you want to use (e.g., Groq or OpenAI).
- Set up a Python environment with necessary dependencies.

---
### Code Walkthrough
Below is the implementation to define and use an AI agent for answering queries.

### Step 0 : Required installation and import of dependencies

In [1]:
!pip install phidata groq duckdb duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.9/716.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 13.9 MB/s eta 0:00:00


#### imports

In [2]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.model.openai import OpenAIChat
from phi.tools.duckduckgo import DuckDuckGo
from phi.storage.agent.json import JsonFileAgentStorage
from datetime import datetime
from phi.tools.csv_tools import CsvTools

### Step 1 : Load the CSV file (Data) and initialise the tool

#### Mount your GDrive to access the file

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Initalise the CSV tool with the required data file.
The CSV tool handles the data file as a table and provides **SQL** features, which can be used by the agent. It handles as a simple 'in-memory' DB

In [4]:
# Initialise the tool with specific data file
CSV_Tool = CsvTools(csvs=['/content/drive/My Drive/Outskill_Agentic_AI/Doc/RainFall.csv'],
                    read_csvs=False)


### Step 2 : Define Agent with CSV tool access

Define an agent with LLM linked. Also the CSV data base is associated to it and parameters set to search for for information from the DB

In [5]:
from google.colab import userdata
import os

# Set your Groq API key or any other LLM API key
# os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

Data_Analyst = Agent (
                        # model=Groq (id="llama3-70b-8192"),
                        model=OpenAIChat (id="gpt-4o-mini"),
                        name = 'Data Analyst',
                        agent_id = 'Data Analyst',
                        description = 'You are an expert data analyst with deep knowledge in statistics and very good at SQL',
                        add_history_to_messages=True,
                        role = 'Expert data analyst with Statistical expertise',
                        storage=JsonFileAgentStorage("./tmp/agent_sessions_json"),
                        tools=[CSV_Tool],
                        instructions=["You have access to databases through the tools",
                                      "Use your SQL expertise to analyse, manipulate data",
                                      "use efficient SQL queries to get the data.",
                                      "Respond to the question prcisely"],
                        show_tool_calls=True,
                        markdown=True,
                      )


### Step 3 : Launch agent in interactive mode

In [6]:
prompt = 'x'

while (prompt != 'exit'):
  prompt = input ("Enter your query ... or 'exit'")
  if prompt != 'exit':

    Res = Data_Analyst.run (prompt)
    print (Res.content)

Enter your query ... or 'exit'Tell me about the data base you have

Running:
 - list_csv_files()

The database consists of a single CSV file named **RainFall**. If you need more specific information about the columns and data contained within this file, please let me know!
Enter your query ... or 'exit'how many rows and columns


INFO     Reading columns from file: RainFall

INFO     Loading csv file: RainFall

INFO     Running query: SELECT COUNT(*) AS row_count FROM RainFall


Running:
 - get_columns(csv_name=RainFall)


Running:
 - query_csv_file(csv_name=RainFall, sql_query=SELECT COUNT(*) AS row_count FROM RainFall)

The **RainFall** dataset contains:

- **Columns**: 15 
  - (SUBDIVISION, YEAR, JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, DEC, ANNUAL)
  
- **Rows**: 4,116 

If you have any further questions or need specific analyses, feel free to ask!
Enter your query ... or 'exit'how many states are there ?


...

INFO     Loading csv file: RainFall

INFO     Running query: SELECT SUBDIVISION, ANNUAL FROM RainFall WHERE ANNUAL IS NOT NULL ORDER BY ANNUAL DESC     
         LIMIT 10

INFO     Loading csv file: RainFall

INFO     Running query: SELECT SUBDIVISION, ANNUAL FROM RainFall WHERE ANNUAL IS NOT NULL AND ANNUAL != 'NA' ORDER 
         BY ANNUAL DESC LIMIT 1


Running:
 - query_csv_file(csv_name=RainFall, sql_query=SELECT SUBDIVISION, ANNUAL FROM RainFall WHERE ANNUAL IS NOT NULL ORDER BY ANNUAL DESC LIMIT 1)


Running:
 - query_csv_file(csv_name=RainFall, sql_query=SELECT SUBDIVISION, ANNUAL FROM RainFall WHERE ANNUAL IS NOT NULL ORDER BY ANNUAL DESC LIMIT 5)

It appears that the records with available annual average rainfall values in the **RainFall** dataset are showing as **NA** (not available) for the top entries I queried.

Let me perform a broader analysis to find a valid state with a non-null annual average value.
Running:
 - query_csv_file(csv_name=RainFall, sql_query=SELECT SUBDIVISION, ANNUAL FROM RainFall WHERE ANNUAL IS NOT NULL ORDER BY ANNUAL DESC LIMIT 10)


Running:
 - query_csv_file(csv_name=RainFall, sql_query=...)

The state with the highest available annual average rainfall in the **RainFall** dataset is **Bihar** with an annual average of **999.9** mm.

If you need more information or further analysis, feel free to ask

In [ ]:
## Current time and date
from datetime import datetime
Now = datetime.now ()
Today = Now.strftime ("%d-%b-%Y")

# Define an internet search agent
Explorer = Agent (
                      # model=Groq (id="llama3-70b-8192"),
                      model=OpenAIChat (id="gpt-4o-mini"),
                      name = 'Explorer',
                      agent_id = 'Explorer',
                      description = 'You are good at gathering important inforamtion from internet for given topic',
                      add_history_to_messages=True,
                      role = 'Expert internet information explorer',
                      storage=JsonFileAgentStorage("./tmp/agent_sessions_json"),
                      tools=[DuckDuckGo()],
                      instructions=["Get relevant information from internet.",
                                    "Don't make up anything.",
                                    "Provide your summary also.",
                                    "Consider you are on "+Today],
                      # show_tool_calls=True,
                      markdown=True,
                  )

# Add to the team of Data analyst
Data_Analyst = Agent (
                        # model=Groq (id="llama3-70b-8192"),
                        model=OpenAIChat (id="gpt-4o-mini"),
                        name = 'Data Analyst',
                        agent_id = 'Data Analyst',
                        description = 'You are an expert data analyst with deep knowledge in statistics and very good at SQL',
                        add_history_to_messages=True,
                        role = 'Expert data analyst with Statistical expertise',
                        storage=JsonFileAgentStorage("./tmp/agent_sessions_json"),
                        tools=[CSV_Tool],
                        team=[Explorer],
                        instructions=["You have access to databases through the tools",
                                      "Use your SQL expertise to analyse, manipulate data",
                                      "use efficient SQL queries to get the data.",
                                      "Look for additional information if needed from internet"
                                      "Respond to the question prcisely"],
                        # show_tool_calls=True,
                        markdown=True,
                      )